In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models
from torchvision import datasets
from torchvision import transforms
from torchmetrics.classification import MulticlassAccuracy
from torchinfo import summary
import os.path

In [2]:
no_epochs = 50
learning_rate = 0.001
batch_size = 128

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

acc_function = MulticlassAccuracy(num_classes=102, average='micro').to(device)
loss_fn = nn.CrossEntropyLoss()

PATH = './model.pth'

cuda:0


In [3]:
# Data Augmentation
train_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Assuming you want to keep the default transformations for testing/validation:
default_transforms = transforms.Compose([
    models.VGG16_BN_Weights.IMAGENET1K_V1.transforms()
])

flowers_train = datasets.Flowers102(root='./data', split='train', download=True, transform=train_transforms)
flowers_test = datasets.Flowers102(root='./data', split='test', download=True, transform=default_transforms)
flowers_val = datasets.Flowers102(root='./data', split='val', download=True, transform=default_transforms)


In [4]:
def get_data_loader(batch_size):
    train_loader = torch.utils.data.DataLoader(flowers_train, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(flowers_test, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(flowers_val, batch_size=batch_size, shuffle=True)
    return train_loader, test_loader, val_loader

In [5]:
# Early stopping based on accuracy
class AccuracyEarlyStopper:
    def __init__(self, patience=3, min_delta=0.5):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.max_validation_accuracy = 0

    def early_stop(self, validation_accuracy):
        if validation_accuracy > (self.max_validation_accuracy + self.min_delta):
            self.max_validation_accuracy = validation_accuracy
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [6]:
def train(model, optimizer, dataloader, loss_fn=loss_fn):
    running_loss_value = 0
    for images, labels in dataloader:
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        running_loss_value += loss.item()
        loss.backward()
        optimizer.step()
    return running_loss_value / len(dataloader)

def test_eval(model, dataloader, loss_fn=loss_fn):
    running_loss_value = 0
    running_acc_value = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            acc = acc_function(outputs, labels)
            running_loss_value += loss.item()
            running_acc_value += acc.item()
    running_acc_value /= len(dataloader)
    running_loss_value /= len(dataloader)
    return running_acc_value*100, running_loss_value

def train_eval_test(model, train_dataloader, val_dataloader, test_dataloader, no_epochs=10):
    es = AccuracyEarlyStopper()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train_loss_arr, train_acc_arr, eval_loss_arr, eval_acc_arr = [], [], [], []
    for i in range(no_epochs):
        train_loss = train(model, optimizer, train_dataloader)
        eval_acc, eval_loss = test_eval(model, val_dataloader)
        print(f'Epoch {i+1} Train Loss: {train_loss:>8f}, Eval Accuracy: {eval_acc:>0.2f}%, Eval Loss: {eval_loss:>8f}')
        train_loss_arr.append(train_loss)
        eval_loss_arr.append(eval_loss)
        eval_acc_arr.append(eval_acc)
        if es.early_stop(eval_acc):
            print('Early stopping activated')
            break
    test_acc, test_loss = test_eval(model, test_dataloader)
    print(f"Test Accuracy: {test_acc}, Test Loss: {test_loss}")
    return train_loss_arr, train_acc_arr, eval_loss_arr, eval_acc_arr, test_acc, test_loss

In [21]:
def create_VGG_model():
    model = models.vgg16_bn(weights=models.VGG16_BN_Weights.DEFAULT)
    new_classifier_head = nn.Sequential(
        nn.Linear(25088, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, 102)
    )
    
    for param in model.parameters():
        param.requires_grad = False
        
    model.classifier = new_classifier_head

    return model.to(device=device)

def create_resnet_model():
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    
    new_fc = nn.Linear(2048, 102)
    
    for param in model.parameters():
        param.requires_grad = False
        
    model.fc = new_fc
    
    return model.to(device)

def create_efficientnet_model(size):
    match size:
        case 's':
            model = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.DEFAULT)
        case 'm':
            model = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.DEFAULT)
        case 'l':
            model = models.efficientnet_v2_l(weights=models.EfficientNet_V2_L_Weights.DEFAULT)
        case _:
            model = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.DEFAULT)



    for param in model.parameters():
        param.requires_grad = False
        
    new_classifier = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(1280, 102)
    )
    
    model.classifier = new_classifier
    
    return model.to(device)

In [8]:
# if os.path.isfile(PATH):
#     #model.load_state_dict(torch.load(PATH))
#     print("Model loaded")
train_data_loader, test_data_loader, val_data_loader = get_data_loader(batch_size)

In [9]:
VGG_model = create_VGG_model()
print("VGG16")
VGG_train_acc, VGG_train_loss, VGG_eval_acc, VGG_eval_loss, VGG_test_acc, VGG_test_loss = train_eval_test(
    VGG_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

In [10]:
#torch.save(model.state_dict(), PATH)

In [13]:
ResNet_model = create_resnet_model()
print("RESNET")
ResNet_train_acc, ResNet_rain_loss, ResNet_eval_acc, ResNet_eval_loss, ResNet_test_acc, ResNet_test_loss = train_eval_test(
    ResNet_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

Epoch 1 Train Loss: 4.573868, Eval Accuracy: 20.09%, Eval Loss: 4.311918
Epoch 2 Train Loss: 4.164435, Eval Accuracy: 46.95%, Eval Loss: 3.979304
Epoch 3 Train Loss: 3.832032, Eval Accuracy: 61.18%, Eval Loss: 3.673984
Epoch 4 Train Loss: 3.521262, Eval Accuracy: 70.58%, Eval Loss: 3.395507
Epoch 5 Train Loss: 3.251028, Eval Accuracy: 74.61%, Eval Loss: 3.139261
Epoch 6 Train Loss: 2.938107, Eval Accuracy: 75.29%, Eval Loss: 2.909765
Epoch 7 Train Loss: 2.693691, Eval Accuracy: 78.24%, Eval Loss: 2.680166
Epoch 8 Train Loss: 2.510690, Eval Accuracy: 78.72%, Eval Loss: 2.511398
Epoch 9 Train Loss: 2.274873, Eval Accuracy: 81.19%, Eval Loss: 2.313196
Epoch 10 Train Loss: 2.089021, Eval Accuracy: 81.75%, Eval Loss: 2.160114
Epoch 11 Train Loss: 1.966115, Eval Accuracy: 81.98%, Eval Loss: 2.024163
Epoch 12 Train Loss: 1.813929, Eval Accuracy: 81.76%, Eval Loss: 1.893847
Epoch 13 Train Loss: 1.690214, Eval Accuracy: 83.14%, Eval Loss: 1.788872
Epoch 14 Train Loss: 1.521371, Eval Accuracy: 8

In [19]:
efficientnet_s_model = create_efficientnet_model('s')

print("EFFICIENTNET")

EN_train_acc, EN_rain_loss, EN_eval_acc, EN_eval_loss, EN_test_acc, EN_test_loss = train_eval_test(
    efficientnet_s_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

EFFICIENTNET
Epoch 1 Train Loss: 4.585929, Eval Accuracy: 12.16%, Eval Loss: 4.341773
Epoch 2 Train Loss: 4.213454, Eval Accuracy: 30.76%, Eval Loss: 4.043918
Epoch 3 Train Loss: 3.882857, Eval Accuracy: 44.59%, Eval Loss: 3.756024
Epoch 4 Train Loss: 3.575458, Eval Accuracy: 54.21%, Eval Loss: 3.495055
Epoch 5 Train Loss: 3.267345, Eval Accuracy: 60.60%, Eval Loss: 3.251611
Epoch 6 Train Loss: 3.049254, Eval Accuracy: 63.51%, Eval Loss: 3.010157
Epoch 7 Train Loss: 2.791121, Eval Accuracy: 65.77%, Eval Loss: 2.820816
Epoch 8 Train Loss: 2.572379, Eval Accuracy: 68.34%, Eval Loss: 2.624314
Epoch 9 Train Loss: 2.403833, Eval Accuracy: 70.09%, Eval Loss: 2.461561
Epoch 10 Train Loss: 2.233039, Eval Accuracy: 71.49%, Eval Loss: 2.354933
Epoch 11 Train Loss: 2.097642, Eval Accuracy: 71.68%, Eval Loss: 2.222022
Epoch 12 Train Loss: 1.994350, Eval Accuracy: 71.75%, Eval Loss: 2.108337
Epoch 13 Train Loss: 1.793482, Eval Accuracy: 74.84%, Eval Loss: 1.992619
Epoch 14 Train Loss: 1.680784, Eva

In [22]:
efficientnet_m_model = create_efficientnet_model('m')

print("EFFICIENTNET")

EN_train_acc, EN_rain_loss, EN_eval_acc, EN_eval_loss, EN_test_acc, EN_test_loss = train_eval_test(
    efficientnet_m_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

Downloading: "https://download.pytorch.org/models/efficientnet_v2_m-dc08266a.pth" to /home/UG/chan0965/.cache/torch/hub/checkpoints/efficientnet_v2_m-dc08266a.pth
11.3%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

26.4%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

41.2%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config vari

Epoch 1 Train Loss: 4.611276, Eval Accuracy: 7.74%, Eval Loss: 4.437325
Epoch 2 Train Loss: 4.296187, Eval Accuracy: 19.90%, Eval Loss: 4.187956
Epoch 3 Train Loss: 4.050389, Eval Accuracy: 30.40%, Eval Loss: 3.978835
Epoch 4 Train Loss: 3.808729, Eval Accuracy: 38.92%, Eval Loss: 3.767979
Epoch 5 Train Loss: 3.584365, Eval Accuracy: 45.21%, Eval Loss: 3.583624
Epoch 6 Train Loss: 3.373561, Eval Accuracy: 48.36%, Eval Loss: 3.398142
Epoch 7 Train Loss: 3.177260, Eval Accuracy: 50.38%, Eval Loss: 3.260493
Epoch 8 Train Loss: 2.989014, Eval Accuracy: 52.56%, Eval Loss: 3.108575
Epoch 9 Train Loss: 2.855293, Eval Accuracy: 53.64%, Eval Loss: 2.990578
Epoch 10 Train Loss: 2.672568, Eval Accuracy: 54.73%, Eval Loss: 2.854990
Epoch 11 Train Loss: 2.537781, Eval Accuracy: 58.14%, Eval Loss: 2.739861
Epoch 12 Train Loss: 2.405982, Eval Accuracy: 57.55%, Eval Loss: 2.621988
Epoch 13 Train Loss: 2.283607, Eval Accuracy: 58.04%, Eval Loss: 2.554203
Epoch 14 Train Loss: 2.233719, Eval Accuracy: 59

In [23]:
efficientnet_l_model = create_efficientnet_model('l')

print("EFFICIENTNET")

EN_train_acc, EN_rain_loss, EN_eval_acc, EN_eval_loss, EN_test_acc, EN_test_loss = train_eval_test(
    efficientnet_l_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

Downloading: "https://download.pytorch.org/models/efficientnet_v2_l-59c71312.pth" to /home/UG/chan0965/.cache/torch/hub/checkpoints/efficientnet_v2_l-59c71312.pth
5.2%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

11.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

18.6%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config varia

EFFICIENTNET
Epoch 1 Train Loss: 4.650917, Eval Accuracy: 14.02%, Eval Loss: 4.113809
Epoch 2 Train Loss: 3.842600, Eval Accuracy: 40.39%, Eval Loss: 3.440678
Epoch 3 Train Loss: 3.176581, Eval Accuracy: 61.27%, Eval Loss: 2.860436
Epoch 4 Train Loss: 2.636687, Eval Accuracy: 70.56%, Eval Loss: 2.365478
Epoch 5 Train Loss: 2.201291, Eval Accuracy: 75.59%, Eval Loss: 1.980211
Epoch 6 Train Loss: 1.832341, Eval Accuracy: 79.69%, Eval Loss: 1.680065
Epoch 7 Train Loss: 1.583375, Eval Accuracy: 82.16%, Eval Loss: 1.471907
Epoch 8 Train Loss: 1.376850, Eval Accuracy: 82.45%, Eval Loss: 1.304898
Epoch 9 Train Loss: 1.203705, Eval Accuracy: 85.98%, Eval Loss: 1.167222
Epoch 10 Train Loss: 1.110708, Eval Accuracy: 86.97%, Eval Loss: 1.073971
Epoch 11 Train Loss: 0.950422, Eval Accuracy: 87.28%, Eval Loss: 0.991237
Epoch 12 Train Loss: 0.890768, Eval Accuracy: 86.98%, Eval Loss: 0.945972
Epoch 13 Train Loss: 0.854771, Eval Accuracy: 88.53%, Eval Loss: 0.870441
Epoch 14 Train Loss: 0.769708, Eva